# This notebook is a simple solution to the cartpole balancing game using 2 methods

## 1) Random Search:
This method simply creates many random agents and chooses the best one. Because our problem is very simple, this method works well enough. If the problem is more complex, it will need more parameters that define its agents; this would, therefore, make random search not efficient as most of the random agents will not be solving our problem well enough.

## 2) Random ascent:
This method is also very simple, but it still has the dimensionality problem (complex problems are hard to solve using this method). This method works by starting with a random agent (you could even run random search on a number of agents and use the best one to start the random ascent). Then, you pick random numbers to add to your current agent's parameters. If the agent improves, keep the change; otherwise, neglect it.

### The cartpole balancing game
I used the OpenAI's gym environment to run the game. The game is very simple. The player controls a cart that he/she can move to the left or to the right. The player should try to balance a pole that is standing vertically on the cart without the pole going out of control and falling.
<img src="img/cartpole_game.png">
The game is considered solved if, according to OpenAI's documentation, "when the average reward is greater than or equal to 195.0 over 100 consecutive trials". I will try to accomplish it in the least time.

In [1]:
#Importing dependencies
import numpy as np
import gym
import time

env = gym.make('CartPole-v0') # Creating the openAI environment

In [2]:
'''
This function run an agent in an environment until the episode (game) terminates.

Args:
    env: OpenAI's env object
    parameters: numpy array of paramters that describe the agent
    
Returns:
    The total reward after the episode terminates
'''
def run_episode(env,parameters):
    observation = env.reset() # Get initial observations when the evironment starts
    total_reward = 0
    for _ in range(200):
        action = 0 if np.matmul(parameters,observation) < 0 else 1 # The agents decision is a simple linear combination of observations and paramters
        observation, reward, done, info = env.step(action)
        total_reward += reward
        if done:
            break
    return total_reward

'''
Runs n episodes (using run_episode) to try to minimize variance in the measurement of rewards
Args:
    n: number of episodes to run
    env: OpenAI's env object
    parameters: numpy array of paramters that describe the agent

Returns:
    The total reward averaged over n episodes
'''
def run_episodes(n,env,parameters):
    total_reward = 0
    for _ in range(n):
        total_reward += run_episode(env,parameters)
    return total_reward/n

### First we will try to use random search

In [3]:
'''
Runs random search to find the best agent
Args:
    num_episodes: The number of episodes used in run_episodes to estimate the fitness of an agent
    max_reward: The maximum reward an agent can achieve
    print_info: If True, info about the best agent and its training is printed

Returns:
    best_agent: Parameters of the best agent found
    best_agent_eval: The fitness of the best agent after running it for 100 episodes (The initial goal was to have a fitness > 195 for 100 episodes)
    running_time: The running time of the random search algorithm
    num_iterations: The number of random agents tested before the best one is found
'''
def random_search(num_episodes=8,max_reward=200.0,print_info=False):
    best_parameters = None
    best_reward = 0
    num_iterations = 0
    initial_time = time.time()
    for _ in range(10000):
        num_iterations += 1
        parameters = np.random.rand(4)*2 - 1 # Random agent
        reward = run_episodes(num_episodes,env,parameters) # A measure of quality of the agent
        if reward>best_reward:
            best_reward = reward
            best_parameters = parameters
        if best_reward == max_reward:
            break
    end_time = time.time()
    running_time = (end_time-initial_time)*1000
    best_agent_eval = run_episodes(100,env,best_parameters)
    
    if print_info:
        print("Time of execution: {} ms".format(running_time))
        print("Found best agent after {} iterations, each ran the game for {} episodes => ran the game {} times.".format(num_iterations,num_episodes,num_iterations*num_episodes))
        print("Agent Evaluation: {}".format(best_agent_eval))
    
    return best_parameters,best_agent_eval,running_time,num_iterations

num_episodes = 4 # Seems to be the smallest number to achieve an average total reward of at least 195
num_tests = 100

avg_fitness = 0
avg_run_time = 0
avg_iters = 0
for _ in range(num_tests):
    _,fitness,run_time,iters = random_search(num_episodes)
    avg_fitness += fitness
    avg_run_time += run_time
    avg_iters += iters

avg_fitness /= num_tests
avg_run_time /= num_tests
avg_iters /= num_tests

print("Average Execution Time: {}".format(avg_run_time))
print("Average agent fitness: {}".format(avg_fitness))
print("Finding best agent takes {} iterations, each ran the game for {} episodes => ran the game on average {} times.".format(avg_iters,num_episodes,avg_iters*num_episodes))

Average Execution Time: 28.429315090179443
Average agent fitness: 197.72530000000003
Finding best agent takes 24.83 iterations, each ran the game for 4 episodes => ran the game on average 99.32 times.


### Next, We will try random ascent to see the difference

In [4]:
noise_scaling = 0.1 # The amount of randomness to try to add at each iteration
best_parameters_ascent = (np.random.rand(4)*2 - 1) # Initial random agent
best_reward = 0
max_reward = 200.0
initial_time = time.time()
for _ in range(10000):
    parameters = best_parameters_ascent + (np.random.rand(4)*2 - 1)*noise_scaling # Add random numbers to the current agent
    reward = run_episodes(10,env,parameters)
    if reward>best_reward: # If the agent improved, keep it; otherwise, neglect it.
        best_reward = reward
        best_parameters_ascent = parameters
    if best_reward == max_reward:
        break

end_time = time.time()
print(best_reward)
print("Time of execution: {} ms".format((end_time-initial_time)*1000))

10.2
Time of execution: 6060.2171421051025 ms


### Running the best agent on the game

In [5]:
observation = env.reset()
timestep = 0
for _ in range(200):
    best_action = 0 if np.matmul(best_parameters_ascent,observation) < 0 else 1
    observation, reward, done, info = env.step(best_action)
    env.render()
    if done:
        break
